In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [7]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [9]:
df = pd.read_csv('Monthly Exchange Rates.csv')

In [11]:
df_melted = df.melt(id_vars = ['Year', 'Variables'],
                    value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                    var_name = 'Month',
                    value_name='Value'

)

In [13]:
df_melted['Value'].fillna(0, inplace=True)

In [15]:
month_map = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
            'Jul': '07', 'Aug': '08', 'Sept': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [17]:
df_melted['Month'] = df_melted['Month'].map(month_map)

In [19]:
df_melted["Month"].fillna(1, inplace=True) 

In [21]:
df_melted

,Year,Variables,Month,Value
0,2025,Inter-Bank Exchange Rate - End Period (GHC/US$),01,15.3000
1,2025,Inter-Bank Exchange Rate - Month Average (GHC/...,01,14.9720
2,2025,Inter-Bank Exchange Rate - End Period (GHC/GBP),01,19.0003
3,2025,Inter-Bank Exchange Rate - Month Average (GHC/...,01,18.4715
4,2025,Inter-Bank Exchange Rate - End Period (GHC/EURO),01,15.9012
...,...,...,...,...
499,2019,Inter-Bank Exchange Rate - Month Average (GHC/...,12,5.5377
500,2019,Inter-Bank Exchange Rate - End Period (GHC/GBP),12,7.3164
501,2019,Inter-Bank Exchange Rate - Month Average (GHC/...,12,7.2610
502,2019,Inter-Bank Exchange Rate - End Period (GHC/EURO),12,6.2114


In [23]:
df_melted['Year'] = df_melted['Year'].astype(int)
df_melted['Month'] = df_melted['Month'].astype(int)

In [65]:
#df_melted['Month'].fillna('Jan', inplace=True)

In [25]:
df_melted.isnull().sum()

Year         0
Variables    0
Month        0
Value        0
dtype: int64

In [27]:
df_melted['Date'] = pd.to_datetime(df_melted['Year'].astype(str) + '-' + df_melted['Month'].astype(str), format='%Y-%m')

In [29]:
df_melted

,Year,Variables,Month,Value,Date
0,2025,Inter-Bank Exchange Rate - End Period (GHC/US$),1,15.3000,2025-01-01
1,2025,Inter-Bank Exchange Rate - Month Average (GHC/...,1,14.9720,2025-01-01
2,2025,Inter-Bank Exchange Rate - End Period (GHC/GBP),1,19.0003,2025-01-01
3,2025,Inter-Bank Exchange Rate - Month Average (GHC/...,1,18.4715,2025-01-01
4,2025,Inter-Bank Exchange Rate - End Period (GHC/EURO),1,15.9012,2025-01-01
...,...,...,...,...,...
499,2019,Inter-Bank Exchange Rate - Month Average (GHC/...,12,5.5377,2019-12-01
500,2019,Inter-Bank Exchange Rate - End Period (GHC/GBP),12,7.3164,2019-12-01
501,2019,Inter-Bank Exchange Rate - Month Average (GHC/...,12,7.2610,2019-12-01
502,2019,Inter-Bank Exchange Rate - End Period (GHC/EURO),12,6.2114,2019-12-01


In [43]:
#df_melted.drop(columns=['Year', 'Month'], inplace=True)

In [31]:
df_melted.sort_values('Date', inplace=True)

In [33]:
df_melted.reset_index(drop=True, inplace=True)

In [35]:
print(df_melted.head(20))

    Year                                          Variables  Month   Value  \
0   2019    Inter-Bank Exchange Rate - End Period (GHC/GBP)      1  6.5412   
1   2019    Inter-Bank Exchange Rate - End Period (GHC/GBP)      1  6.5121   
2   2019  Inter-Bank Exchange Rate - Month Average (GHC/...      1  4.8947   
3   2019    Inter-Bank Exchange Rate - End Period (GHC/US$)      1  4.9506   
4   2019  Inter-Bank Exchange Rate - Month Average (GHC/...      1  5.5868   
5   2019    Inter-Bank Exchange Rate - End Period (GHC/US$)      1  5.3161   
6   2019  Inter-Bank Exchange Rate - Month Average (GHC/...      1  5.3067   
7   2019  Inter-Bank Exchange Rate - Month Average (GHC/...      1  6.3142   
8   2019  Inter-Bank Exchange Rate - Month Average (GHC/...      1  6.5527   
9   2019   Inter-Bank Exchange Rate - End Period (GHC/EURO)      1  5.7956   
10  2019  Inter-Bank Exchange Rate - Month Average (GHC/...      1  5.8417   
11  2019   Inter-Bank Exchange Rate - End Period (GHC/EURO)     

In [37]:
print(df_melted.shape)

(504, 5)


In [45]:
df_melted.to_csv('Exchange_Rates.csv', index=False)

print("DataFrame successfully saved to Exchange_Rate.csv")

DataFrame successfully saved to Exchange_Rate.csv


In [47]:
exchange = pd.read_csv('Exchange_Rates.csv', parse_dates=['Date'])

In [49]:
exchange.sort_values('Date', inplace=True)

In [51]:
exchange['Quarter'] = exchange['Date'].dt.quarter
exchange['DayofYear'] = exchange['Date'].dt.dayofyear

In [53]:
exchange

,Year,Variables,Month,Value,Date,Quarter,DayofYear
0,2019,Inter-Bank Exchange Rate - End Period (GHC/GBP),1,6.5412,2019-01-01,1,1
1,2019,Inter-Bank Exchange Rate - End Period (GHC/GBP),1,6.5121,2019-01-01,1,1
2,2019,Inter-Bank Exchange Rate - Month Average (GHC/...,1,4.8947,2019-01-01,1,1
3,2019,Inter-Bank Exchange Rate - End Period (GHC/US$),1,4.9506,2019-01-01,1,1
4,2019,Inter-Bank Exchange Rate - Month Average (GHC/...,1,5.5868,2019-01-01,1,1
...,...,...,...,...,...,...,...
498,2025,Inter-Bank Exchange Rate - End Period (GHC/GBP),12,0.0000,2025-12-01,4,335
499,2025,Inter-Bank Exchange Rate - Month Average (GHC/...,12,0.0000,2025-12-01,4,335
500,2025,Inter-Bank Exchange Rate - End Period (GHC/EURO),12,0.0000,2025-12-01,4,335
501,2025,Inter-Bank Exchange Rate - Month Average (GHC/...,12,0.0000,2025-12-01,4,335


In [55]:
exchange_grouped = exchange.groupby(["Date", "Year", "Month", "Quarter", "DayofYear", "Variables"], as_index=False).agg({"Value": "mean"})
exchange_pivot = exchange_grouped.pivot(index=["Date", "Year", "Month", "Quarter", "DayofYear"], 
                                        columns="Variables", 
                                        values="Value").reset_index()

exchange_pivot.columns.name = None  # Remove MultiIndex in columns

In [57]:
exchange_pivot.columns

Index(['Date', 'Year', 'Month', 'Quarter', 'DayofYear',
       'Inter-Bank Exchange Rate - End Period (GHC/EURO)',
       'Inter-Bank Exchange Rate - End Period (GHC/GBP)',
       'Inter-Bank Exchange Rate - End Period (GHC/US$)',
       'Inter-Bank Exchange Rate - Month Average (GHC/EURO)',
       'Inter-Bank Exchange Rate - Month Average (GHC/GBP)',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$)'],
      dtype='object')

In [59]:
# Create lag features (for each currency)


lag_features = ['Inter-Bank Exchange Rate - Month Average (GHC/US$)', 'Inter-Bank Exchange Rate - End Period (GHC/EURO)' ,'Inter-Bank Exchange Rate - End Period (GHC/GBP)']
for col in lag_features:
    exchange_pivot[f'{col} lag 1'] = exchange_pivot[col].shift(1)
    exchange_pivot[f'{col} lag 3'] = exchange_pivot[col].shift(3)
    exchange_pivot[f'{col} lag 6'] = exchange_pivot[col].shift(6)

In [61]:
# Create moving averages

for col in lag_features:
    exchange_pivot[f"{col} MA 3"] = exchange_pivot[col].rolling(window=3).mean()
    exchange_pivot[f"{col} MA 6"] = exchange_pivot[col].rolling(window=6).mean()
    exchange_pivot[f"{col} MA 12"] = exchange_pivot[col].rolling(window=12).mean()

In [63]:
# Create percentage change (momentum features)

for col in lag_features:
    exchange_pivot[f"{col} Change 1M"] = exchange_pivot[col].pct_change(1)
    exchange_pivot[f"{col} Change 3M"] = exchange_pivot[col].pct_change(3)
    exchange_pivot[f"{col} Change 6M"] = exchange_pivot[col].pct_change(6)

In [65]:
# Seasonal binary features

exchange_pivot['Is_Start_Of_Year'] = (exchange_pivot['Month']==1).astype(int)
exchange_pivot['Is_End_Of_Year'] = (exchange_pivot['Month']==12).astype(int)

In [67]:
exchange_pivot.isnull().sum()

Date                                                             0
Year                                                             0
Month                                                            0
Quarter                                                          0
DayofYear                                                        0
Inter-Bank Exchange Rate - End Period (GHC/EURO)                 0
Inter-Bank Exchange Rate - End Period (GHC/GBP)                  0
Inter-Bank Exchange Rate - End Period (GHC/US$)                  0
Inter-Bank Exchange Rate - Month Average (GHC/EURO)              0
Inter-Bank Exchange Rate - Month Average (GHC/GBP)               0
Inter-Bank Exchange Rate - Month Average (GHC/US$)               0
Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 1         1
Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 3         3
Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 6         6
Inter-Bank Exchange Rate - End Period (GHC/EURO) lag 1        

In [69]:
exchange_pivot.dropna(inplace=True)

In [71]:
exchange_pivot.isnull().sum()

Date                                                            0
Year                                                            0
Month                                                           0
Quarter                                                         0
DayofYear                                                       0
Inter-Bank Exchange Rate - End Period (GHC/EURO)                0
Inter-Bank Exchange Rate - End Period (GHC/GBP)                 0
Inter-Bank Exchange Rate - End Period (GHC/US$)                 0
Inter-Bank Exchange Rate - Month Average (GHC/EURO)             0
Inter-Bank Exchange Rate - Month Average (GHC/GBP)              0
Inter-Bank Exchange Rate - Month Average (GHC/US$)              0
Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 1        0
Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 3        0
Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 6        0
Inter-Bank Exchange Rate - End Period (GHC/EURO) lag 1          0
Inter-Bank

In [73]:
exchange_pivot.to_csv('processed_exchange_rates.csv', index=False)

In [75]:
exchange_pivot.shape

(58, 40)

In [77]:
print(exchange_pivot.columns)

Index(['Date', 'Year', 'Month', 'Quarter', 'DayofYear',
       'Inter-Bank Exchange Rate - End Period (GHC/EURO)',
       'Inter-Bank Exchange Rate - End Period (GHC/GBP)',
       'Inter-Bank Exchange Rate - End Period (GHC/US$)',
       'Inter-Bank Exchange Rate - Month Average (GHC/EURO)',
       'Inter-Bank Exchange Rate - Month Average (GHC/GBP)',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$)',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 1',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 3',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 6',
       'Inter-Bank Exchange Rate - End Period (GHC/EURO) lag 1',
       'Inter-Bank Exchange Rate - End Period (GHC/EURO) lag 3',
       'Inter-Bank Exchange Rate - End Period (GHC/EURO) lag 6',
       'Inter-Bank Exchange Rate - End Period (GHC/GBP) lag 1',
       'Inter-Bank Exchange Rate - End Period (GHC/GBP) lag 3',
       'Inter-Bank Exchange Rate - End Period (GHC/GBP) lag 6

# training. 

In [79]:
target_variables = [
    "Inter-Bank Exchange Rate - End Period (GHC/EURO)",
    "Inter-Bank Exchange Rate - End Period (GHC/GBP)",
    "Inter-Bank Exchange Rate - End Period (GHC/US$)"
]


# Exclude non-numeric and target columns from features
X = exchange_pivot.drop(columns=['Date', 'Year', 'Month', 'Quarter', 'DayofYear'] + target_variables)
y = exchange_pivot[target_variables]

***Since exchange rates are time-based, we’ll use 2024 as training data and 2025 as test data:***

In [81]:
train_data = exchange_pivot[exchange_pivot['Year'] < 2025]
test_data = exchange_pivot[exchange_pivot['Year'] == 2025]

X_train = train_data.drop(columns=['Date', 'Year', 'Month', 'Quarter', 'DayofYear'] + target_variables)
y_train = train_data[target_variables]

X_test = test_data.drop(columns=['Date', 'Year', 'Month', 'Quarter', 'DayofYear'] + target_variables)
y_test = test_data[target_variables]

In [83]:
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state = 42))

In [85]:
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [87]:
y_pred = model.predict(X_test)

In [89]:
y_pred

array([[10.245839 , 11.748766 ,  9.3613775],
       [15.2803365, 18.200909 , 13.834618 ],
       [ 9.315804 , 10.6847485,  8.507133 ]])

In [91]:
# Compute metrics for each target variable


for i, col in enumerate(target_variables):
    mae = mean_absolute_error(y_test[col], y_pred[:, i])
    rmse = mean_squared_error(y_test[col], y_pred[:, i], squared=False)
    
    print(f"{col} -> MAE: {mae:.4f}, RMSE: {rmse:.4f}")

Inter-Bank Exchange Rate - End Period (GHC/EURO) -> MAE: 4.1610, RMSE: 5.5622
Inter-Bank Exchange Rate - End Period (GHC/GBP) -> MAE: 4.7603, RMSE: 6.3518
Inter-Bank Exchange Rate - End Period (GHC/US$) -> MAE: 3.9713, RMSE: 5.1047


***feature engineering for improvement***

In [93]:
# Get feature importance from trained model

feature_importance = np.mean([tree.feature_importances_ for tree in model.estimators_], axis=0)


# Create DataFrame and sort by importance
feature_importance_df = pd.DataFrame({"Feature": X_train.columns, "Importance": feature_importance})
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)

print(feature_importance_df.head(20))  # Top 20 important features

                                              Feature  Importance
0   Inter-Bank Exchange Rate - Month Average (GHC/...    0.246458
2   Inter-Bank Exchange Rate - Month Average (GHC/...    0.197839
1   Inter-Bank Exchange Rate - Month Average (GHC/...    0.197117
9   Inter-Bank Exchange Rate - End Period (GHC/GBP...    0.034581
5   Inter-Bank Exchange Rate - Month Average (GHC/...    0.033014
6   Inter-Bank Exchange Rate - End Period (GHC/EUR...    0.031970
13  Inter-Bank Exchange Rate - Month Average (GHC/...    0.029080
20  Inter-Bank Exchange Rate - End Period (GHC/GBP...    0.027273
12  Inter-Bank Exchange Rate - Month Average (GHC/...    0.025393
19  Inter-Bank Exchange Rate - End Period (GHC/GBP...    0.023914
16  Inter-Bank Exchange Rate - End Period (GHC/EUR...    0.021641
3   Inter-Bank Exchange Rate - Month Average (GHC/...    0.020263
7   Inter-Bank Exchange Rate - End Period (GHC/EUR...    0.017662
18  Inter-Bank Exchange Rate - End Period (GHC/GBP...    0.017600
4   Inter-

***Instead of using all features, let’s select the best ones:***

In [111]:
#selector = SelectKBest(f_regression, k=15)
#X_train_selected = selector.fit_transform(X_train, y_train)
#X_test_selected = selector.transform(X_test)


#selected_features = X_train.columns[selector.get_support()]
#print('Selected Features:', selected_features)

In [97]:
selected_features_dict = {}

for target_col in y_train.columns:
    selector = SelectKBest(score_func=f_regression, k=15)
    selector.fit(X_train, y_train[target_col])

    selected_features = X_train.columns[selector.get_support()]
    selected_features_dict[target_col] = selected_features.tolist()

    print(f"Top features for {target_col}:")
    print(selected_features)
    print()

Top features for Inter-Bank Exchange Rate - End Period (GHC/EURO):
Index(['Inter-Bank Exchange Rate - Month Average (GHC/EURO)',
       'Inter-Bank Exchange Rate - Month Average (GHC/GBP)',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$)',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$) lag 1',
       'Inter-Bank Exchange Rate - End Period (GHC/EURO) lag 1',
       'Inter-Bank Exchange Rate - End Period (GHC/GBP) lag 1',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$) MA 3',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$) MA 6',
       'Inter-Bank Exchange Rate - Month Average (GHC/US$) MA 12',
       'Inter-Bank Exchange Rate - End Period (GHC/EURO) MA 3',
       'Inter-Bank Exchange Rate - End Period (GHC/EURO) MA 6',
       'Inter-Bank Exchange Rate - End Period (GHC/EURO) MA 12',
       'Inter-Bank Exchange Rate - End Period (GHC/GBP) MA 3',
       'Inter-Bank Exchange Rate - End Period (GHC/GBP) MA 6',
       'Inter-Bank Exchange Rate - 

In [240]:
#selected_features = []
#for i in range(y_train.shape[1]):  # Loop over target variables
 #   selector = SelectKBest(f_regression, k=15)
  #  X_train_selected = selector.fit_transform(X_train, y_train.iloc[:, i])
   # selected_features.append(X_train_selected)

# Convert selected features back to a NumPy array (optional)
#X_train_selected = np.hstack(selected_features)
#X_test_selected = selector.transform(X_test)

# Get selected feature names
#selected_features = X_train.columns[selector.get_support()]
#print("Selected Features:", selected_features)

In [105]:
models = {}
y_preds = {}

for target_col in y_train.columns:
    selected_features = selected_features_dict[target_col]


  # Train model using only the selected features
    model = XGBRegressor(n_estimators=200, learning_rate =0.5, max_depth=6, random_state=42)
    model.fit(X_train[selected_features], y_train[target_col])

    # Predict on test set using same selected features
    y_pred = model.predict(X_test[selected_features])


    # Store the trained model and predictions
    models[target_col] = model
    y_preds[target_col] = y_pred

In [107]:
y_pred

array([ 7.661087 , 14.188952 ,  5.6842995], dtype=float32)

In [109]:
for target_col in y_train.columns:
    y_true = y_test[target_col]
    y_pred = y_preds[target_col]

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    print(f"{target_col} -> MAE: {mae:.4f}, RMSE: {rmse:.4f}")

Inter-Bank Exchange Rate - End Period (GHC/EURO) -> MAE: 2.2804, RMSE: 3.6304
Inter-Bank Exchange Rate - End Period (GHC/GBP) -> MAE: 2.7411, RMSE: 3.9883
Inter-Bank Exchange Rate - End Period (GHC/US$) -> MAE: 2.3455, RMSE: 3.3719
